In [1]:
import numpy as np
import torch
import torch.nn as nn
import time
from Utilities import Utilities as Util
from IPython.display import clear_output

In [2]:
class ResidualLayer(nn.Module):
    def __init__(self, filters, kernal_size=3):
        super().__init__()

        self.conv2d_sequential = nn.Sequential(                
            nn.Conv2d(filters, filters, kernal_size, padding=(kernal_size - 1) // 2),
            nn.BatchNorm2d(filters),
            nn.ReLU(),
            nn.Conv2d(filters, filters, kernal_size, padding=(kernal_size - 1) // 2),
            nn.BatchNorm2d(filters),
        )

        self.relu = nn.ReLU()

    def forward(self, x):
        residual = x
        x = self.conv2d_sequential(x)
        x += residual
        x = self.relu(x)

        return x
    
class ConvolutionLayer(nn.Module):
    def __init__(self, infilters, outfilters, kernal_size=3):
        super().__init__()
        
        self.conv2d_sequential = nn.Sequential(                
            nn.Conv2d(infilters, outfilters, kernal_size, padding=(kernal_size - 1) // 2),
            nn.BatchNorm2d(outfilters),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.conv2d_sequential(x)
        return x
    
class PolicyHead(nn.Module):
    def __init__(self, filters):
        super().__init__()
        self.filters = filters

        self.head = nn.Sequential(
            nn.Conv2d(self.filters, 1, 1),
            nn.Flatten(),
            nn.BatchNorm1d(225),
            nn.ReLU(),
            nn.Linear(225, 225)
        )

    def forward(self, x):
        x = self.head(x)
        return x

class NeuralNetwork(nn.Module):
    def __init__(self, filters, feature_dimensions, residual_layers=5, kernal_size=3):
        super().__init__()

        self.conv_layer = ConvolutionLayer(feature_dimensions, filters, kernal_size=kernal_size)
        self.residual_layers = nn.ModuleList([ResidualLayer(filters, kernal_size=kernal_size) for _ in range(residual_layers)])
        self.policy_head = PolicyHead(filters)

    def forward(self, x):      
        x = self.conv_layer(x)
        for layer in self.residual_layers:
            x = layer(x)
        x = self.policy_head(x)
        
        return x
    
Filters = 128
Layers = 20
HistoryDepth = 8
KernalSize = 5

model = NeuralNetwork(Filters, HistoryDepth + 1, Layers, kernal_size=KernalSize)
#model = torch.compile(model)

In [ ]:
model.load_state_dict(torch.load('../Models/HumanModels/128f20l5kAllBut715e/14.pt'))
model.eval()

In [ ]:
# LEGACY
def makeMove(gamestate, x, y):
    turn = gamestate[0, 0, 0]
    whiteHistory = []
    blackHistory = []

    if HistoryDepth == 1:
        if turn:
            gamestate[2, x, y] = True
        else:
            gamestate[1, x, y] = True
        gamestate[0, :, :] = not turn
        return gamestate

    for i in range(HistoryDepth):
        whiteHistory.append(gamestate[i + 1].copy())
        blackHistory.append(gamestate[i + HistoryDepth + 1].copy())
    currentWhite = whiteHistory[0].copy()
    currentBlack = blackHistory[0].copy()

    if turn:
        currentBlack[x, y] = True
    else:
        currentWhite[x, y] = True
    
    gamestate[0, :, :] = not turn
    gamestate[1] = currentWhite
    gamestate[HistoryDepth + 1] = currentBlack

    for i in range(1, HistoryDepth):
        gamestate[i + 1] = whiteHistory[i - 1]
    for i in range(1, HistoryDepth):
        gamestate[i + HistoryDepth + 1] = blackHistory[i - 1]

    return gamestate

In [ ]:
board = np.zeros((HistoryDepth + 1, 15, 15), dtype=bool)
board[0, :, :] = True

In [ ]:
while (True):
    modelInput = torch.from_numpy(board.astype(np.float32)).unsqueeze(0)
    with torch.no_grad():
        output = model(modelInput)
    output = torch.nn.functional.softmax(output, dim=1)
    output = np.array(output)
    output = output.reshape(15, 15)
    output *= ~board[1]
    output *= ~board[HistoryDepth + 1]
    output = output.flatten()

    index = output.argmax()
    x, y = int(index // 15), int(index % 15)

    board = makeMove(board, x, y)
    clear_output(wait=True)
    print(Util.sliceGamestate(board, HistoryDepth, 6))
    time.sleep(1)

    x = int(input("X:"))
    y = int(input("Y:"))

    board = makeMove(board, x - 1, 14 - y + 1)
    clear_output(wait=True)
    print(Util.sliceGamestate(board, HistoryDepth, 6))
    time.sleep(1)